In [7]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    LogitsProcessorList,
    MinLengthLogitsProcessor,
    StoppingCriteriaList,
    MaxLengthCriteria,
)
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

model.generation_config.pad_token_id = model.generation_config.eos_token_id

maxLength = 30

prompt = "Today I believe we can finally"

# return Pytorch tensor
inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs.input_ids

In [8]:
# Greedy Search
greedy_output = model.generate(
    input_ids, 
    max_new_tokens=maxLength, 
    return_dict_in_generate=True, 
    output_scores=True
)

# Beam Search
beam_output = model.generate(
    input_ids, 
    max_new_tokens=maxLength, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    early_stopping=True,
    return_dict_in_generate=True, 
    output_scores=True
)

# Top-K
# set top_k to 50
top_k_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_new_tokens=maxLength, 
    top_k=50,
    return_dict_in_generate=True, 
    output_scores=True
)

# Top-P (nuclear sampling)

# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
top_p_output = model.generate(
    input_ids,
    do_sample=True, 
    max_new_tokens=maxLength, 
    top_k=50, 
    top_p=0.95, 
    return_dict_in_generate=True, 
    output_scores=True
)



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

### Print outputs

In [9]:
print(f'Greedy Search',tokenizer.batch_decode(greedy_output[0], skip_special_tokens=True))
print(f'\nBeam Search',tokenizer.batch_decode(beam_output[0], skip_special_tokens=True))
print(f'\nTop-K',tokenizer.batch_decode(top_k_output[0], skip_special_tokens=True))
print(f'\nTop-P',tokenizer.batch_decode(top_p_output[0], skip_special_tokens=True))

Greedy Search ['Today I believe we can finally get to the point where we can make a difference in the lives of the people of the United States of America.\n\nI believe that we can']

Beam Search ['Today I believe we can finally get to the bottom of this issue.\n\n"We need to find a way to make sure that we don\'t get into a situation where we']

Top-K ['Today I believe we can finally make the change for the worse.\n\nI am grateful to every single person who has helped make this possible. I hope and expect I can do']

Top-P ['Today I believe we can finally achieve the full realization of our vision and our desire to reach the ideal of humanity for our planet," said U.N. spokesman Stephane Dujar']


### Generate Scores

In [10]:
input_length = inputs.input_ids.shape[1]
# Greedy
greedy_output_scores = model.compute_transition_scores(
    greedy_output.sequences, greedy_output.scores, normalize_logits=True
)
greedy_output_tokens = greedy_output.sequences[:, input_length:]

# Beam
beam_output_scores = model.compute_transition_scores(
    beam_output.sequences, beam_output.scores, normalize_logits=True
)
beam_output_tokens = beam_output.sequences[:, input_length:]

# Top-K
top_k_output_scores = model.compute_transition_scores(
    top_k_output.sequences, top_k_output.scores, normalize_logits=True
)
top_k_output_tokens = top_k_output.sequences[:, input_length:]

# Top-P
top_p_output_scores = model.compute_transition_scores(
    top_p_output.sequences, top_p_output.scores, normalize_logits=True
)
top_p_output_tokens = top_p_output.sequences[:, input_length:]

# for tok, score in zip(greedy_output_tokens[0], greedy_output_scores[0]):
#     # | token | token string | logits | probability
#     print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.numpy():.3f} | {np.exp(score.numpy()):.2%}")


In [13]:
# Define function for perplexity and likelihood
def calculate_perplexity_and_likelihood(scores):
    # compute the sum of the log-likelihoods
    perplexity = 0
    likelihood = 0
    for score in scores[0]:
        perplexity += score.numpy()
        likelihood += np.exp(perplexity)
    perplexity = perplexity / len(scores[0])    
    return perplexity, likelihood

In [14]:
# calculate the perplexity and likelihood for the greedy output
greedy_perplexity, greedy_likelihood = calculate_perplexity_and_likelihood(greedy_output_scores)
print(f"Greedy perplexity: {greedy_perplexity:.3f}")
print(f"Greedy likelihood: {greedy_likelihood:.2%}")

# calculate the perplexity and likelihood for the beam output
beam_perplexity, beam_likelihood = calculate_perplexity_and_likelihood(beam_output_scores)
print(f"Beam perplexity: {beam_perplexity:.3f}")
print(f"Beam likelihood: {beam_likelihood:.2%}")

# calculate the perplexity and likelihood for the top-k output
top_k_perplexity, top_k_likelihood = calculate_perplexity_and_likelihood(top_k_output_scores)
print(f"Top-k perplexity: {top_k_perplexity:.3f}")
print(f"Top-k likelihood: {top_k_likelihood:.2%}")

# calculate the perplexity and likelihood for the top-p output
top_p_perplexity, top_p_likelihood = calculate_perplexity_and_likelihood(top_p_output_scores)
print(f"Top-p perplexity: {top_p_perplexity:.3f}")
print(f"Top-p likelihood: {top_p_likelihood:.2%}")

Greedy perplexity: -1.346
Greedy likelihood: 7.93%
Beam perplexity: -6.135
Beam likelihood: 7.48%
Top-k perplexity: -2.134
Top-k likelihood: 9.00%
Top-p perplexity: -2.111
Top-p likelihood: 3.69%
